Funciones originales y eficientes

In [ ]:
import numpy as np
class Original():
  #Traslación
  def traslacion(self,P,T):
    P=np.matrix([[P[0]],[P[1]],[P[2]],[1]])
    T=np.matrix([[1,0,0,T[0]],
                [0,1,0,T[1]],
                [0,0,1,T[2]],
                [0,0,0,1]])
    producto=T@P
    res=[]
    for i in range(len(producto)-1):
      res.append(float(producto[i]))
    return res

  #Rotación
  def rotacion(self,c,theta,axis):
    P=np.matrix([[c[0]],[c[1]],[c[2]],[1]])
    theta=(theta*np.pi)/180
    if axis=="x":
      T=np.matrix([[1,0,0,0],
                  [0,np.cos(theta),-1*np.sin(theta),0],
                  [0,np.sin(theta),np.cos(theta),0],
                  [0,0,0,1]])
    elif axis=="y":
      T=np.matrix([[np.cos(theta),0,np.sin(theta),0],
                  [0,1,0,0],
                  [-1*np.sin(theta),0,np.cos(theta),0],
                  [0,0,0,1]])
    elif axis=="z":
      T=np.matrix([[np.cos(theta),-1*np.sin(theta),0,0],
                  [np.sin(theta),np.cos(theta),0,0],
                  [0,0,1,0],
                  [0,0,0,1]])

    producto=T@P
    res=[]
    for i in range(len(producto)-1):
      res.append(float(producto[i]))
    return res

  #Escalación
  def escalacion(self,P,esc):
    P=np.matrix([P[0],P[1],P[2],1])
    S=np.matrix([[esc[0],0,0,0],
                [0,esc[1],0,0],
                [0,0,esc[2],0],
                [0,0,0,1]])
    producto=np.array(P@S)
    producto=(np.asarray(producto)).flatten()
    res=[]
    for i in range(len(producto)-1):
      res.append(float(producto[i]))
    return res

################################################################################
class Eficiente():
  def toVector(self,matriz,option):
    if option=="filas":
      res=[]
      for i in range(len(matriz)):
        for j in range(len(matriz[0])):
          res.append(matriz[i][j])
      return res 
    if option=="columnas":
      res=[]
      for i in range(len(matriz[0])):
        for j in range(len(matriz)):
          res.append(matriz[j][i])
      return res

  def productoE(self,matriz,vector,orden): #Matriz,Vector, Orden de multiplicacion (vector*matriz ó matriz*vector)
    if orden=="MV":
      opcion="filas"
    if orden=="VM":
      opcion="columnas"

    vectorM=toVector(matriz,opcion)
    res=[0]*len(vector)
    aux=0
    index=0

    for i in range(len(vectorM)):
      prueba=(i+1)%len(vector)

      aux+=vectorM[i]*vector[index]
      index+=1

      if i==0:
        prueba=1
      if prueba==0:
        res[int(i/len(vector))]=aux
        aux=0
        index=0
      
    return res
  
  #Traslación eficiente
  def traslacionE(self,P,T):
    P=[P[0],P[1],P[2],1]
    T=[[1,0,0,T[0]],
       [0,1,0,T[1]],
       [0,0,1,T[2]],
       [0,0,0,1]]
    res=self.productoE(T,P,"MV")
    res.pop(-1)
    return res
  
  #Rotación eficiente
  def rotacionE(self,c,theta,axis):
    P=[c[0],c[1],c[2],1]
    theta=(theta*np.pi)/180
    if axis=="x":
      T=[[1,0,0,0],
         [0,np.cos(theta),-1*np.sin(theta),0],
         [0,np.sin(theta),np.cos(theta),0],
         [0,0,0,1]]
    elif axis=="y":
      T=[[np.cos(theta),0,np.sin(theta),0],
         [0,1,0,0],
         [-1*np.sin(theta),0,np.cos(theta),0],
         [0,0,0,1]]
    elif axis=="z":
      T=[[np.cos(theta),-1*np.sin(theta),0,0],
         [np.sin(theta),np.cos(theta),0,0],
         [0,0,1,0],
         [0,0,0,1]]

    res=self.productoE(T,P,"MV")
    res.pop(-1)
    return res

  #Escalación eficiente
  def escalacionE(self,P,esc):
    P=[P[0],P[1],P[2],1]
    S=[[esc[0],0,0,0],
       [0,esc[1],0,0],
       [0,0,esc[2],0],
       [0,0,0,1]]
    res=self.productoE(S,P,"VM")
    res.pop(-1)
    return res

def main():
  #Punto inicial
  punto=[1,2,3]
  print("Punto Original:\n",punto)
  objO=Original()

  #Traslación
  tras=[1,1,1]
  nuevoPunto=objO.traslacion(punto,tras)
  #print("Traslación:\n",nuevoPunto)

  #Rotación
  angulo=90 #angulo en grados
  eje="z"
  nuevoPunto=objO.rotacion(punto,angulo,eje)
  #print("Rotación:\n",nuevoPunto)

  #Escalación
  s=[2,2,2]
  nuevoPunto=objO.escalacion(punto,s)
  print("Escalación:\n",nuevoPunto)

  ##############################################################################
  objE=Eficiente()
  #Traslación eficiente
  tras=[1,1,1]
  nuevoPunto=objE.traslacionE(punto,tras)
  #print("Traslación eficiente:\n",nuevoPunto)

  #Rotación eficiente
  angulo=90
  eje="z"
  nuevoPunto=objE.rotacionE(punto,angulo,eje)
  #print("Rotación eficiente:\n",nuevoPunto)

  #Escalación eficiente
  s=[2,2,2]
  nuevoPunto=objE.escalacionE(punto,s)
  print("Escalación eficiente:\n",nuevoPunto)

if __name__=="__main__":
  main()

Punto Original:
 [1, 2, 3]
Escalación:
 [2.0, 4.0, 6.0]
Escalación eficiente:
 [2, 4, 6]


Raytracer

In [ ]:
from PIL import Image
import numpy as np

w = 400
h = 300

def normalize(x):
    x /= np.linalg.norm(x)
    return x

def intersect_plane(O, D, P, N):
    # Return the distance from O to the intersection of the ray (O, D) with the 
    # plane (P, N), or +inf if there is no intersection.
    # O and P are 3D points, D and N (normal) are normalized vectors.
    denom = np.dot(D, N)
    if np.abs(denom) < 1e-6:
        return np.inf
    d = np.dot(P - O, N) / denom
    if d < 0:
        return np.inf
    return d

def intersect_sphere(O, D, S, R):
    # Return the distance from O to the intersection of the ray (O, D) with the 
    # sphere (S, R), or +inf if there is no intersection.
    # O and S are 3D points, D (direction) is a normalized vector, R is a scalar.
    a = np.dot(D, D)
    OS = O - S
    b = 2 * np.dot(D, OS)
    c = np.dot(OS, OS) - R * R
    disc = b * b - 4 * a * c
    if disc > 0:
        distSqrt = np.sqrt(disc)
        q = (-b - distSqrt) / 2.0 if b < 0 else (-b + distSqrt) / 2.0
        t0 = q / a
        t1 = c / q
        t0, t1 = min(t0, t1), max(t0, t1)
        if t1 >= 0:
            return t1 if t0 < 0 else t0
    return np.inf

def intersect(O, D, obj):
    if obj['type'] == 'plane':
        return intersect_plane(O, D, obj['position'], obj['normal'])
    elif obj['type'] == 'sphere':
        return intersect_sphere(O, D, obj['position'], obj['radius'])

def get_normal(obj, M):
    # Find normal.
    if obj['type'] == 'sphere':
        N = normalize(M - obj['position'])
    elif obj['type'] == 'plane':
        N = obj['normal']
    return N
    
def get_color(obj, M):
    color = obj['color']
    if not hasattr(color, '__len__'):
        color = color(M)
    return color

def trace_ray(rayO, rayD):
    # Find first point of intersection with the scene.
    t = np.inf
    for i, obj in enumerate(scene):
        t_obj = intersect(rayO, rayD, obj)
        if t_obj < t:
            t, obj_idx = t_obj, i
    # Return None if the ray does not intersect any object.
    if t == np.inf:
        return
    # Find the object.
    obj = scene[obj_idx]
    # Find the point of intersection on the object.
    M = rayO + rayD * t
    # Find properties of the object.
    N = get_normal(obj, M)
    color = get_color(obj, M)
    toL = normalize(L - M)
    toO = normalize(O - M)
    # Shadow: find if the point is shadowed or not.
    l = [intersect(M + N * .0001, toL, obj_sh) 
            for k, obj_sh in enumerate(scene) if k != obj_idx]
    if l and min(l) < np.inf:
        return
    # Start computing the color.
    col_ray = ambient
    # Lambert shading (diffuse).
    col_ray += obj.get('diffuse_c', diffuse_c) * max(np.dot(N, toL), 0) * color
    # Blinn-Phong shading (specular).
    col_ray += obj.get('specular_c', specular_c) * max(np.dot(N, normalize(toL + toO)), 0) ** specular_k * color_light
    return obj, M, N, col_ray

def add_sphere(position, radius, color):
    return dict(type='sphere', position=np.array(position), 
        radius=np.array(radius), color=np.array(color), reflection=.5)
    
def add_plane(position, normal):
    return dict(type='plane', position=np.array(position), 
        normal=np.array(normal),
        color=lambda M: (color_plane0 
            if (int(M[0] * 2) % 2) == (int(M[2] * 2) % 2) else color_plane1),
        diffuse_c=.75, specular_c=.5, reflection=.25)
    
################################################################################
class Eficiente():
  def toVector(self,matriz,option):
    if option=="filas":
      res=[]
      for i in range(len(matriz)):
        for j in range(len(matriz[0])):
          res.append(matriz[i][j])
      return res 
    if option=="columnas":
      res=[]
      for i in range(len(matriz[0])):
        for j in range(len(matriz)):
          res.append(matriz[j][i])
      return res

  def productoE(self,matriz,vector,orden): #Matriz,Vector, Orden de multiplicacion (vector*matriz ó matriz*vector)
    if orden=="MV":
      opcion="filas"
    if orden=="VM":
      opcion="columnas"

    vectorM=self.toVector(matriz,opcion)
    res=[0]*len(vector)
    aux=0
    index=0

    for i in range(len(vectorM)):
      prueba=(i+1)%len(vector)

      aux+=vectorM[i]*vector[index]
      index+=1

      if i==0:
        prueba=1
      if prueba==0:
        res[int(i/len(vector))]=aux
        aux=0
        index=0
      
    return res
  
  #Traslación eficiente
  def traslacionE(self,P,T):
    P=[P[0],P[1],P[2],1]
    T=[[1,0,0,T[0]],
       [0,1,0,T[1]],
       [0,0,1,T[2]],
       [0,0,0,1]]
    res=self.productoE(T,P,"MV")
    res.pop(-1)
    return res
  
  #Rotación eficiente
  def rotacionE(self,c,theta,axis):
    P=[c[0],c[1],c[2],1]
    theta=(theta*np.pi)/180
    if axis=="x":
      T=[[1,0,0,0],
         [0,np.cos(theta),-1*np.sin(theta),0],
         [0,np.sin(theta),np.cos(theta),0],
         [0,0,0,1]]
    elif axis=="y":
      T=[[np.cos(theta),0,np.sin(theta),0],
         [0,1,0,0],
         [-1*np.sin(theta),0,np.cos(theta),0],
         [0,0,0,1]]
    elif axis=="z":
      T=[[np.cos(theta),-1*np.sin(theta),0,0],
         [np.sin(theta),np.cos(theta),0,0],
         [0,0,1,0],
         [0,0,0,1]]

    res=self.productoE(T,P,"MV")
    res.pop(-1)
    return res

  #Escalación eficiente
  def escalacionE(self,P,esc):
    P=[P[0],P[1],P[2],1]
    S=[[esc[0],0,0,0],
       [0,esc[1],0,0],
       [0,0,esc[2],0],
       [0,0,0,1]]
    res=self.productoE(S,P,"VM")
    res.pop(-1)
    return res

# List of objects.
color_plane0 = 1. * np.ones(3)
color_plane1 = 0. * np.ones(3)

objE=Eficiente()
# Puntos iniciales
sphere1=[0,.1,1] 
sphere2=[-1, .1, 2.25]
sphere3=[-2.75, .1, 3.5]
# Datos de transformacion
T=[1,1,1]
angulo,eje=90,"y"
escalar=[2,2,2]

pasosX=np.linspace(0,T[0],90)
pasosY=np.linspace(0,T[1],90)
pasosZ=np.linspace(0,T[2],90)

angulos=np.linspace(0,angulo,90)

escalacionesX=np.linspace(escalar[0],0,90)
escalacionesY=np.linspace(escalar[1],0,90)
escalacionesZ=np.linspace(escalar[2],0,90)

for i in range(len(pasosX)):
  name=str(i)
  # Traslacion
  traslacion=[pasosX[i],pasosY[i],pasosZ[i]] 
  pointi1=objE.traslacionE(sphere1,traslacion)
  #print(pointi1)
  
  # Rotación a 90° sobre eje z
  pointi2=objE.rotacionE(sphere2,angulos[i],eje)
  #print(pointi2)

  # Escalación
  esc=[escalacionesX[i],escalacionesY[i],escalacionesZ[i]]
  pointi3=objE.escalacionE(sphere3,esc)
  #print(pointi3)

  scene = [add_sphere(pointi1, .6, [0., 0., 1.]),
          add_sphere(pointi2, .6, [.5, .223, .5]),
          add_sphere(pointi3, .6, [1., .572, .184]),
          add_plane([0., -.5, 0.], [0., 1., 0.]),
      ]

  # Light position and color.
  L = np.array([5., 5., -10.])
  color_light = np.ones(3)

  # Default light and material parameters.
  ambient = .05
  diffuse_c = 1.
  specular_c = 1.
  specular_k = 50

  depth_max = 5  # Maximum number of light reflections.
  col = np.zeros(3)  # Current color.
  O = np.array([0., 0.35, -1.])  # Camera.
  Q = np.array([0., 0., 0.])  # Camera pointing to.
  img = np.zeros((h, w, 3))

  r = float(w) / h
  # Screen coordinates: x0, y0, x1, y1.
  S = (-1., -1. / r + .25, 1., 1. / r + .25)

  # Loop through all pixels.
  for i, x in enumerate(np.linspace(S[0], S[2], w)):
      if i % 10 == 0:
          #print(i / float(w) * 100, "%")
          procentaje=True
      for j, y in enumerate(np.linspace(S[1], S[3], h)):
          col[:] = 0
          Q[:2] = (x, y)
          D = normalize(Q - O)
          depth = 0
          rayO, rayD = O, D
          reflection = 1.
          # Loop through initial and secondary rays.
          while depth < depth_max:
              traced = trace_ray(rayO, rayD)
              if not traced:
                  break
              obj, M, N, col_ray = traced
              # Reflection: create a new ray.
              rayO, rayD = M + N * .0001, normalize(rayD - 2 * np.dot(rayD, N) * N)
              depth += 1
              col += reflection * col_ray
              reflection *= obj.get('reflection', 1.)
          img[h - j - 1, i, :] = np.clip(col, 0, 1)

  im = Image.fromarray((255 * img).astype(np.uint8), "RGB")
  print("Terminó frame: ",name)
  im.save(name+".png")